# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [6]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random

In [7]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=2000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [8]:
# iterate all datasets in the eval_data folder
runners = {}
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=0)
    
results = []

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [9]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "prompt": prompt, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)

In [10]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        scores = runner.run_basic_prompting_experiment(prompt=prompt, max_iters=5)
        update_results("basic_prompting", dataset, scores, {"prompt": prompt})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: mushroom_1.json
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
Total score: 14.2 (accuracy: 4.0, completeness: 4.0, fluency: 2.2, conciseness: 4.0)
--
Prompt: You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.
Total score: 13.8 (accuracy: 4.0, completeness: 3.6, fluency: 2.2, conciseness: 4.0)
--
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.
Total score: 14.6 (accuracy: 4.0, completeness: 4.0, fluency: 2.6, conciseness: 4.0)
--
=====


## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [11]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(max_iters=5, n_few_shot=i, prompt=prompts[0])
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: mushroom_1.json
Few-shot n: 1
Total score: 15.0 (accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 4.0)
--
Few-shot n: 3
Total score: 15.0 (accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 4.0)
--
Few-shot n: 5
Total score: 15.0 (accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 4.0)
--
=====
Dataset: pdf_1.json
Few-shot n: 1
Total score: 14.6 (accuracy: 3.2, completeness: 4.0, fluency: 3.4, conciseness: 4.0)
--
Few-shot n: 3
Total score: 14.8 (accuracy: 3.2, completeness: 4.0, fluency: 3.6, conciseness: 4.0)
--
Few-shot n: 5
Total score: 15.0 (accuracy: 3.2, completeness: 4.0, fluency: 3.8, conciseness: 4.0)
--
=====
Dataset: student_1.json
Few-shot n: 1
Total score: 14.8 (accuracy: 4.0, completeness: 3.6, fluency: 3.2, conciseness: 4.0)
--
Few-shot n: 3
Total score: 13.8 (accuracy: 4.0, completeness: 2.8, fluency: 3.0, conciseness: 4.0)
--
Few-shot n: 5
Total score: 14.8 (accuracy: 4.0, completeness: 3.6, fluency: 3.2, conciseness: 4.0)
--
=

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [12]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i, j in [[0, 3], [0, 5], [3, 3], [3, 5]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(max_iters=5, n_labeled_few_shot=i, n_bootstrapped_few_shot=j)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: mushroom_1.json
Few-shot n: 0, Bootstrapped n: 3


100%|██████████| 17/17 [01:56<00:00,  6.85s/it]


Bootstrapped 1 full traces after 17 examples in round 0.


100%|██████████| 17/17 [00:00<00:00, 216.52it/s]


Bootstrapped 1 full traces after 17 examples in round 0.


...

Bootstrapped 4 full traces after 18 examples in round 0.


100%|██████████| 18/18 [00:00<00:00, 192.48it/s]


Bootstrapped 4 full traces after 18 examples in round 0.


100%|██████████| 18/18 [00:00<00:00, 208.07it/s]


Bootstrapped 4 full traces after 18 examples in round 0.
Total score: 15.2 (accuracy: 4.0, completeness: 4.0, fluency: 3.2, conciseness: 4.0)
--
=====


In [13]:
result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df

,dataset,prompt,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot
0,mushroom_1.json,You are helping users understand an ML model's...,14.2,4.0,4.0,2.2,4.0,NaN,NaN
1,mushroom_1.json,You are helping users who do not have experien...,13.8,4.0,3.6,2.2,4.0,NaN,NaN
2,mushroom_1.json,You are helping users understand an ML model's...,14.6,4.0,4.0,2.6,4.0,NaN,NaN
3,pdf_1.json,You are helping users understand an ML model's...,15.0,3.2,4.0,3.8,4.0,NaN,NaN
4,pdf_1.json,You are helping users who do not have experien...,13.8,3.2,3.2,3.4,4.0,NaN,NaN
5,pdf_1.json,You are helping users understand an ML model's...,14.4,3.2,4.0,3.2,4.0,NaN,NaN
6,student_1.json,You are helping users understand an ML model's...,15.0,4.0,4.0,3.0,4.0,NaN,NaN
7,student_1.json,You are helping users who do not have experien...,14.6,4.0,3.6,3.0,4.0,NaN,NaN
8,student_1.json,You are helping users understand an ML model's...,15.0,4.0,4.0,3.0,4.0,NaN,NaN
9,mushroom_1.json,You are helping users understand an ML model's...,15.0,4.0,4.0,3.0,4.0,1.0,NaN
